In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 

/Users/catherineyeh/.local/share/virtualenvs/story-viz-mNgXibik/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [3]:
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [4]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=3000, api_key=os.getenv("ANTHROPIC_API_KEY"))

# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [5]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters. Make sure the formatting matches the markers in the actual text, after the table of contents. Keep all punctuation marks and capitalization. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [132]:
story_info = {
   "title": "Whispers of the Tea Route",
  "type": "Book",
  "author": "ChatGPT",
  "year": 2024,
  "image": "covers/whispers.png",
  }

In [133]:
# read in txt file from scripts folder
story_name = "whispers"
analysis_type = "theme" # character, theme
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [134]:
if analysis_type == "theme":
    story_name += "_themes"

In [135]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]

In [136]:
# # split story into chunks
# char_chunk = 120000
# print(len(story))
# story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
# print(len(story_chunks))

In [137]:
# use full story
story_chunks = [story]

In [138]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

Chunk 0


In [139]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
markers = []
for res in all_res:
    markers.extend(res.markers)

markers

["Chapter 1: The Merchant's Daughter",
 'Chapter 2: Caravan of Secrets',
 'Chapter 3: An Encounter in the Desert',
 'Chapter 4: Bandits in the Night',
 'Chapter 5: The Town of Lost Souls',
 'Chapter 6: The Scroll of Secrets',
 'Chapter 7: An Oath to Complete the Journey',
 'Chapter 8: The Shadow of Betrayal',
 'Chapter 9: Trials of Loyalty',
 'Chapter 10: The Unraveling',
 'Chapter 11: The Battle of Trust',
 'Chapter 12: The Crossroads of Destiny']

In [480]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers

In [78]:
# # remove any marker that doesn't include one of these words
# exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [140]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")
with open(f"json/{story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [141]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [142]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [143]:
# read in markers from summary.json
with open(f"json/{story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [144]:
story_formatted[-100:] 

'hrough to its end, and with the hope of peace guiding them, they would fulfill her father s legacy. '

In [145]:
story_formatted.find(last_line)
last_line

"Together, they would see this mission through to its end, and with the hope of peace guiding them, they would fulfill her father's legacy."

In [146]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    # print("marker", marker)
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    # print("next_marker", next_marker)
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line
    # print("formatted_last_line", formatted_last_line)

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    # print("cur_last_line", repr(cur_last_line))
    # print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))
    first_line_index = story_formatted.find(cur_first_line, cur_min_index)
    # print("first_line_index", first_line_index)

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    # print("last_line_index", last_line_index)
    
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

In [147]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter Chapter 1: The Merchant's Daughter
first_line: LINE 1: The morning sun filtered through thin paper screens, casting warm patterns
LINE 2: across Me
last_line: rvant, drinking in every detail. The world felt vast, and she was
LINE 96: finally stepping into it.


Chapter Chapter 2: Caravan of Secrets
first_line: LINE 1: The caravan was an ever-moving beast, each trader, guard, and servant another
LINE 2: muscle
last_line: d overheard. The road ahead was full of mysteries, and she
LINE 127: was determined to unravel them.


Chapter Chapter 3: An Encounter in the Desert
first_line: LINE 1: The endless dunes stretched out before Mei, golden and unforgiving under the
LINE 2: relentl
last_line: 79: ready to meet them, her heart steady with the strength she drew from those
LINE 180: around her.


Chapter Chapter 4: Bandits in the Night
first_line: LINE 1: Night settled heavily over the desert camp, shadows stretching long and thin
LINE 2: beneath
last_line: INE 153: prize. And when he 

In [148]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{story_name}"):
    os.makedirs(f"chapters/{story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter Chapter 1: The Merchant's Daughter saved.
Chapter Chapter 2: Caravan of Secrets saved.
Chapter Chapter 3: An Encounter in the Desert saved.
Chapter Chapter 4: Bandits in the Night saved.
Chapter Chapter 5: The Town of Lost Souls saved.
Chapter Chapter 6: The Scroll of Secrets saved.
Chapter Chapter 7: An Oath to Complete the Journey saved.
Chapter Chapter 8: The Shadow of Betrayal saved.
Chapter Chapter 9: Trials of Loyalty saved.
Chapter Chapter 10: The Unraveling saved.
Chapter Chapter 11: The Battle of Trust saved.
Chapter Chapter 12: The Crossroads of Destiny saved.


## analyze scene

In [149]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [150]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [151]:
def get_scene_info(chapters, chapter_num):
    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = llm3.invoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = scene_llm.invoke(cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }
        formatted_chapter_scenes.append(formatted_scene)
        formatted_chapter = cur_chapter.replace("\n", " ")
        # replace weird quotes too
        formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [152]:
def get_scene_details(scenes, formatted_chapter_scenes):
    all_scene_details = []
    for i, scene in enumerate(scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        if len(scene_text) == 0:
            all_scene_details.append([])
            continue
        # get scene details for each scene
        cur_scene = scene
        invalid_input = True
        # if model_type == 'gemini':
        #     prompt = get_scenedetails_prompt(cur_scene)
        #     scene_details = llm.invoke(prompt)
        #     print(scene_details)
        #     scene_details = format_gemini_response(scene_details)
        #     print(scene_details)
        #     scene_details = DotDict(scene_details)
        # else:
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = scene_details_llm_theme.invoke(cur_scene)
                else:
                    scene_details = scene_details_llm.invoke(cur_scene)
                invalid_input = False
                # print(scene_details)
            except:
                invalid_input = True
        all_scene_details.append(scene_details)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.characters if analysis_type == "character" else cur_scene_details.themes
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote
            }
            formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [153]:
# get scene info for each chapter
for i, marker in enumerate(markers):
    print(f"Chapter {marker} start.")
    scenes, formatted_chapter_scenes = get_scene_info(chapters, i)
    print(f"Chapter {marker} scenes done.")
    formatted_chapter_scenes = get_scene_details(scenes, formatted_chapter_scenes)
    print(f"Chapter {marker} details done.")

    # remove scenes with no text
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
    # remove characters with no quote
    for scene in formatted_chapter_scenes:
        scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
        # print(scene)
    
    # remove scenes with no characters
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(formatted_chapter_scenes, f, indent=4)

    print(f"Chapter {marker} done.\n")

Chapter Chapter 1: The Merchant's Daughter start.
FIRST LINE: 1
LAST LINE: 32

FIRST LINE: 33
LAST LINE: 52

FIRST LINE: 53
LAST LINE: 76

FIRST LINE: 77
LAST LINE: 96

Chapter Chapter 1: The Merchant's Daughter scenes done.
Chapter Chapter 1: The Merchant's Daughter details done.
Chapter Chapter 1: The Merchant's Daughter done.

Chapter Chapter 2: Caravan of Secrets start.
FIRST LINE: 1
LAST LINE: 32

FIRST LINE: 38
LAST LINE: 72

FIRST LINE: 73
LAST LINE: 101

FIRST LINE: 102
LAST LINE: 127

Chapter Chapter 2: Caravan of Secrets scenes done.
Chapter Chapter 2: Caravan of Secrets details done.
Chapter Chapter 2: Caravan of Secrets done.

Chapter Chapter 3: An Encounter in the Desert start.
FIRST LINE: 1
LAST LINE: 27

FIRST LINE: 28
LAST LINE: 53

FIRST LINE: 54
LAST LINE: 87

FIRST LINE: 88
LAST LINE: 113

FIRST LINE: 148
LAST LINE: 180

Chapter Chapter 3: An Encounter in the Desert scenes done.
Chapter Chapter 3: An Encounter in the Desert details done.
Chapter Chapter 3: An Encount

In [154]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

Chapter 1: The Merchant's Daughter
Chapter 2: Caravan of Secrets
Chapter 3: An Encounter in the Desert
Chapter 4: Bandits in the Night
Chapter 5: The Town of Lost Souls
Chapter 6: The Scroll of Secrets
Chapter 7: An Oath to Complete the Journey
Chapter 8: The Shadow of Betrayal
Chapter 9: Trials of Loyalty
Chapter 10: The Unraveling
Chapter 11: The Battle of Trust
Chapter 12: The Crossroads of Destiny


In [155]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

In [156]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [157]:
len(all_characters), len(all_locations)

(150, 59)

In [158]:
all_characters

{'Desire for freedom': "Mei's longing to join her father",
 'Father-daughter relationship': "Zhang's protective nature towards Mei",
 'Tradition vs. ambition': "Mei's struggle between societal expectations and her dreams",
 'Supportive friendship': "Lin's encouragement and understanding of Mei's desires",
 'Parental Control': "Mei struggles against her father's overprotectiveness",
 'Self-Identity': 'Mei seeks to assert her independence',
 'Friendship Support': 'Lin provides emotional support to Mei',
 'dreams and aspirations': "Mei's longing for freedom",
 'family bonds': 'father-daughter relationship',
 'change and uncertainty': 'unexpected journey ahead',
 'Coming of Age': 'Mei begins to see the world differently and takes on new responsibilities.',
 'Family Bonds': 'The relationship between Mei and her father is highlighted as they work together.',
 'Tradition': 'The market represents a long-standing family tradition that Mei is now a part of.',
 'Journey Ahead': "Represents the be

In [159]:
all_locations

["Mei's room",
 'outer courtyard',
 'family room',
 'bustling tea market',
 'city gates',
 'desert caravan camp',
 'Caravan camp at night',
 "Mei's tent at night",
 'Gobi Desert',
 'desert caravan',
 'oasis camp',
 'desert oasis',
 'desert camp',
 'desert camp at night',
 'desert camp after attack',
 'desert hideout',
 'remote town between dunes',
 'city streets at dusk',
 'narrow alleys of town',
 'quiet corner campsite',
 'darkened streets',
 'dimly lit shop',
 'town gathering',
 'town edge',
 "Mei's bedroom at night",
 'desert caravan encampment',
 'Campfire at night',
 'camp at night',
 'Dawn on the horizon',
 'desert at night',
 'mountains caravan journey',
 'mountain caravan',
 'mountains at night',
 'mountains at dawn',
 'mountain path',
 'campfire gathering',
 'sheltered valley camp',
 'campfire at night',
 'camp near mountains',
 'dense forest',
 'narrow mountain pass',
 'desert plateau',
 'Caravan journey',
 'mountains',
 'the plains',
 'plains at sunset',
 'distant horizon',

In [160]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [161]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [162]:
output

AIMessage(content='{\n    "Desire for freedom": ["Mei\'s longing to join her father", "dreams and aspirations", "Mei\'s longing for freedom"],\n    "Father-daughter relationship": ["Zhang\'s protective nature towards Mei", "family bonds", "Father-Daughter Bond"],\n    "Tradition vs. ambition": ["Mei\'s struggle between societal expectations and her dreams", "Tradition"],\n    "Supportive friendship": ["Lin\'s encouragement and understanding of Mei\'s desires", "Friendship Support", "Friendship"],\n    "Parental Control": ["Mei struggles against her father\'s overprotectiveness"],\n    "Self-Identity": ["Mei seeks to assert her independence", "self-discovery", "identity"],\n    "Change and uncertainty": ["unexpected journey ahead", "Journey Ahead"],\n    "Coming of Age": ["Mei begins to see the world differently and takes on new responsibilities."],\n    "Journey and Adventure": ["Represents the beginning of Mei\'s adventure", "adventure", "the journey continues"],\n    "Father\'s Guida

In [163]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [164]:
location_output

AIMessage(content='{\n  "Mei\'s house": ["Mei\'s room", "Mei\'s bedroom at night", "family room"],\n  "Courtyard": ["outer courtyard"],\n  "Tea market": ["bustling tea market"],\n  "City": ["city gates", "city streets at dusk", "darkened streets", "dimly lit shop", "town gathering", "town edge", "narrow alleys of town"],\n  "Desert": ["Gobi Desert", "desert plateau", "desert at night"],\n  "Caravan camp": ["desert caravan camp", "Caravan camp at night", "desert caravan", "oasis camp", "desert oasis", "desert camp", "desert camp at night", "desert camp after attack", "desert hideout", "desert caravan encampment", "Caravan journey", "Caravan at dawn", "Caravan gathering place"],\n  "Remote town": ["remote town between dunes"],\n  "Campsite": ["quiet corner campsite", "campfire gathering", "Campfire at night", "camp at night", "by the fire"],\n  "Mountains": ["mountains caravan journey", "mountain caravan", "mountains at night", "mountains at dawn", "mountain path", "narrow mountain pass"

In [165]:
# if model_type == "claude":
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [166]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [167]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [168]:
len(all_characters_new), len(all_locations_new)

(84, 15)

In [169]:
all_characters_new 

{'Desire For Freedom': 2,
 'Father-daughter Relationship': 6,
 'Tradition Vs. Ambition': 2,
 'Supportive Friendship': 4,
 'Parental Control': 1,
 'Self-identity': 4,
 'Change And Uncertainty': 3,
 'Coming Of Age': 1,
 "Father's Guidance": 1,
 'Community Bond': 3,
 'Curiosity And Intrigue': 2,
 'Cultural Diversity': 2,
 'Bravery And Independence': 19,
 'Connection And Anticipation': 7,
 'Danger And Warning': 3,
 'Secrecy': 2,
 'Determination': 8,
 'Resilience': 6,
 'Journey And Adventure': 2,
 'Harsh Environment': 1,
 'Reunion': 1,
 'Mystery': 6,
 'Burden': 2,
 'Secrets Revealed': 2,
 'Responsibility': 4,
 'Inner Conflict': 1,
 'Wisdom And Guidance': 1,
 'Unease': 7,
 'Family Bond': 1,
 'Fear And Courage': 1,
 'Unity In Crisis': 16,
 'Danger And Survival': 1,
 'Despair': 1,
 'Healing': 2,
 'Rage And Determination': 1,
 'Loyalty And Fear': 1,
 'Anticipation Of Battle': 1,
 'Pride And Defeat': 1,
 'Discovery': 1,
 'Comfort': 1,
 'Concern': 1,
 'Curiosity': 1,
 'Alienation': 1,
 'Journey':

In [170]:
all_locations_new

["Mei's House",
 'Courtyard',
 'Tea Market',
 'City',
 'Caravan Camp',
 "Mei's Tent At Night",
 'Desert',
 'Remote Town',
 'Campsite',
 'Horizon',
 'Mountains',
 'Valley',
 'Forest',
 'Plains',
 'Battlefield']

In [171]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [172]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [173]:
# create summary json of each chapter
chapter_summaries = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
        
    with open(f"chapters/{story_name}/{marker}.txt") as f:
        chapter = f.read()

    # get summary of chapter
    scene_summaries = " ".join([scene["summary"] for scene in scenes])
    summary = chapter_llm.invoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }
    chapter_summaries.append(chapter_summary)
    
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
    for scene in scenes:
        chapter = scene["chapter"]
        chapter_summary = chapter_summaries[markers.index(chapter)]
        # add location to chapter summary
        location = scene["location"]
        chapter_summary_locations = chapter_summary["locations"]
        if location not in chapter_summary_locations:
            chapter_summary_locations[location] = 1
        else:
            chapter_summary_locations[location] += 1
        # add importance to chapter summary
        importance = scene["importance"]
        chapter_summary["importance"].append(importance)
        # add conflict to chapter summary
        conflict = scene["conflict"]
        chapter_summary["conflict"].append(conflict)
        # add characters to chapter summary
        characters = scene["characters"]
        for character in characters:
            char = character["name"]
            chapter_summary_characters = chapter_summary["characters"]
            if char not in chapter_summary_characters:
                chapter_summary_characters[char] = 1
            else:
                chapter_summary_characters[char] += 1

# now average importance and conflict for each chapter
for chapter_summary in chapter_summaries:
    if len(chapter_summary["importance"]) == 0:
       print(f"Chapter {chapter_summary['chapter']} has no scenes.")
    chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
    chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

# rank each chapter by importance and conflict
importances = []
conflicts = []
for chapter_summary in chapter_summaries:
    importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
    conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
    
# sort importances
sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
# add importance_rank to each chapter
for k, (j, _) in enumerate(sorted_importances):
    chapter_summaries[markers.index(j)]["importance_rank"] = k+1

# sort conflicts
sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
# add conflict_rank to each chapter
for k, (j, _) in enumerate(sorted_conflicts):
    chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

In [174]:
chapter_summaries

[{'chapter': "Chapter 1: The Merchant's Daughter",
  'scenes': 4,
  'length': 7380,
  'num_lines': 96,
  'summary': "Mei grapples with her feelings of entrapment as she prepares for her father's journey, ultimately finding empowerment during a market visit with him, before Zhang surprises her by announcing she will join him.",
  'importance': 0.8,
  'conflict': 0.53,
  'locations': {"Mei's House": 2, 'Courtyard': 1, 'Tea Market': 1},
  'characters': {'Desire For Freedom': 2,
   'Father-daughter Relationship': 3,
   'Tradition Vs. Ambition': 2,
   'Supportive Friendship': 2,
   'Parental Control': 1,
   'Self-identity': 1,
   'Change And Uncertainty': 1,
   'Coming Of Age': 1},
  'importance_rank': 11,
  'conflict_rank': 7},
 {'chapter': 'Chapter 2: Caravan of Secrets',
  'scenes': 4,
  'length': 10040,
  'num_lines': 127,
  'summary': 'Mei arrives at the city gates with her father, meets Kaveh the merchant, feels a connection, hears caravan stories, and overhears a secret conversation 

In [175]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

In [176]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: str = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [177]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Bravery And Independence': 19,
 'Unity In Crisis': 16,
 'Trust': 11,
 'Hope': 9,
 'Determination': 8,
 'Support And Trust': 8,
 'Connection And Anticipation': 7,
 'Unease': 7,
 'Leadership Development': 7,
 'Father-daughter Relationship': 6,
 'Resilience': 6,
 'Mystery': 6,
 'Supportive Friendship': 4,
 'Self-identity': 4,
 'Responsibility': 4,
 'Change And Uncertainty': 3,
 'Community Bond': 3,
 'Danger And Warning': 3,
 'Legacy': 3,
 'Strength': 3}

In [178]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
4


In [179]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [180]:
all_res = []
used_colors = []
for i, arr in enumerate(character_arr):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        # filter all_json to only include characters in arr
        new_scenes = []
        # for scene in all_json["scenes"]:
        for scene in scenes:
            include = False
            for character in scene["characters"]:
                if character["name"] in arr:
                    include = True
                    break
            if include:
                new_scenes.append(scene)

        # take middle 126000 characters of new_scenes
        new_scenes_str = json.dumps(new_scenes)
        new_scenes_str = new_scenes_str[:126000]

        try:
            # prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list. Make sure each character is assigned a unique color."
            prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list."
            # if len(used_colors) > 0:
            #     prompt += f"\ndon't use any of these colors: {used_colors}"
            res = characterlist_llm.invoke(prompt)
            all_res.append(res)
            # int_list = res.characterlist
            # for j, char in enumerate(int_list):
            #     used_colors.append(char.color)
            print(f"Chunk {i} done.")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

Chunk 0 done.
Chunk 1 done.
Chunk 2 done.
Chunk 3 done.
Chunk 4 done.


In [181]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
4


84

In [182]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)


In [183]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [184]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

duplicate color: rgb(75, 0, 130) 11
duplicate color: rgb(255, 20, 147) 13


In [185]:
colorinfo = {}
for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Bravery And Independence': {'color': 'rgb(255, 99, 71)',
  'explanation': "This vibrant red-orange color symbolizes courage and the desire for freedom, reflecting Mei's journey towards independence."},
 'Unity In Crisis': {'color': 'rgb(30, 144, 255)',
  'explanation': 'This strong blue represents solidarity and trust among the caravan members during challenging times.'},
 'Trust': {'color': 'rgb(124, 252, 0)',
  'explanation': 'This bright green signifies growth and the importance of trust in relationships, especially between Mei and her companions.'},
 'Hope': {'color': 'rgb(255, 215, 0)',
  'explanation': 'This warm gold color embodies optimism and the hope that drives Mei forward in her mission.'},
 'Determination': {'color': 'rgb(255, 69, 0)',
  'explanation': "This intense red color reflects Mei's fierce determination to fulfill her father's mission."},
 'Support And Trust': {'color': 'rgb(255, 20, 147)',
  'explanation': 'This vibrant pink symbolizes the emotional support and 

In [186]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

character_info

84


[{'name': 'Desire For Freedom',
  'quote': 'I could come with you, Father. Help you.',
  'group': 'Emotions',
  'color': '',
  'explanation': ''},
 {'name': 'Father-daughter Relationship',
  'quote': "This journey… it isn't for you, Mei.",
  'group': 'Family',
  'color': 'rgb(255, 192, 203)',
  'explanation': 'This soft pink represents the love and protective nature of the father-daughter bond between Mei and Zhang.'},
 {'name': 'Tradition Vs. Ambition',
  'quote': "A daughter's place was here.",
  'group': 'Social Themes',
  'color': '',
  'explanation': ''},
 {'name': 'Supportive Friendship',
  'quote': "One day, you'll show everyone what you're made of, Mei.",
  'group': 'Emotions',
  'color': 'rgb(255, 105, 180)',
  'explanation': 'This bright pink represents the caring and supportive friendships that help Mei navigate her challenges.'},
 {'name': 'Parental Control',
  'quote': "He's too protective.",
  'group': 'Family',
  'color': '',
  'explanation': ''},
 {'name': 'Self-identit

In [187]:
# in final data, for top characters, replace existing color + explanation with new ones

# read in final_data.json
# with open(f"json/{story_name}/final_data.json") as f:
#     final_data = json.load(f)

# character_names = [char["name"] for char in character_info]

# for i, c in enumerate(final_data["characters"]):
#     # see if matching character in character_info
#     if c["name"] in character_names:
#         char_index = character_names.index(c["name"])
#         final_data["characters"][i]["color"] = character_info[char_index]["color"]
#         final_data["characters"][i]["explanation"] = character_info[char_index]["explanation"]
#     else:
#         # clear color and explanation
#         final_data["characters"][i]["color"] = ""
#         final_data["characters"][i]["explanation"] = ""

# # save final_data.json
# with open(f"json/{story_name}/final_data.json", "w") as f:
#     json.dump(final_data, f, indent=4)

In [188]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [189]:
# # read in scenes from all_json
# story_name = "gatsby_themes"
# with open(f"json/{story_name}/final_data.json") as f:
#     all_json = json.load(f)

In [190]:
# locations = all_json["locations"]
# all_locations_new = [loc["name"] for loc in locations]
# character_info = all_json["characters"]

In [191]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [192]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

15
[{'name': "Mei's House", 'quote': "The morning sun filtered through thin paper screens, casting warm patterns across Mei's room.", 'emoji': '🏡'}, {'name': 'Courtyard', 'quote': 'As the afternoon sun began to fade, Mei found herself wandering the outer courtyard, her steps guided by her restless thoughts.', 'emoji': '🏞️'}, {'name': 'Tea Market', 'quote': 'The next morning, Mei found herself in the bustling tea market, watching her father exchange goods with the vendors.', 'emoji': '🍵'}, {'name': 'City', 'quote': 'The caravan was an ever-moving beast, each trader, guard, and servant another muscle working to propel it forward.', 'emoji': '🏙️'}, {'name': 'Caravan Camp', 'quote': 'That evening, the caravan made camp near a small oasis, a blessing of water and shade in the vast, merciless desert.', 'emoji': '🏕️'}, {'name': "Mei's Tent At Night", 'quote': 'Later that night, as Mei was preparing to sleep, she overheard a conversation between her father and Kaveh.', 'emoji': '🌙'}, {'name': 

In [193]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Whispers of the Tea Route',
 'type': 'Book',
 'author': 'ChatGPT',
 'year': 2024,
 'image': 'covers/whispers.png',
 'num_chapters': 12,
 'num_scenes': 66,
 'num_characters': 84,
 'num_locations': 15,
 'chapters': [{'chapter': "Chapter 1: The Merchant's Daughter",
   'scenes': 4,
   'length': 7380,
   'num_lines': 96,
   'summary': "Mei grapples with her feelings of entrapment as she prepares for her father's journey, ultimately finding empowerment during a market visit with him, before Zhang surprises her by announcing she will join him.",
   'importance': 0.8,
   'conflict': 0.53,
   'locations': {"Mei's House": 2, 'Courtyard': 1, 'Tea Market': 1},
   'characters': {'Desire For Freedom': 2,
    'Father-daughter Relationship': 3,
    'Tradition Vs. Ambition': 2,
    'Supportive Friendship': 2,
    'Parental Control': 1,
    'Self-identity': 1,
    'Change And Uncertainty': 1,
    'Coming Of Age': 1},
   'importance_rank': 11,
   'conflict_rank': 7},
  {'chapter': 'Chapter 2:

In [194]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)